In [1]:
import sys
print(sys.executable)

C:\Users\ghaza\Desktop\Ironhack_Data_Analytics\week_10\project\Book_Recomendation_System\.venv\Scripts\python.exe


In [2]:
from sentence_transformers import SentenceTransformer
print("SBERT is ready ✅")

SBERT is ready ✅


## Loading CSV & sanity checks

In [3]:
import pandas as pd

In [4]:
df= pd.read_csv(r"../data/clean/books_merged_clean.csv")

In [5]:
print("Shape:", df.shape)
df.head()

Shape: (1038, 6)


,title,author,published_year,language,subjects,cover
0,21st century houses: riba award-winning homes,dominic bradbury,2022,english,"domestic architecture, architecture, awards, m...",https://openlibrary.org/images/icons/avatar_bo...
1,architecture china 2020 building with nature j...,l. xiangning,2022,english,"architecture, awards, sustainable architecture...",https://openlibrary.org/images/icons/avatar_bo...
2,cyberarts 2021: international compendium prix ...,markus jandl,2022,english,"computer art, awards, computer animation, prix...",https://covers.openlibrary.org/b/id/13794706-M...
3,"deutsche bank ""artists of the year"" 2021: maxw...",maxwell alexandre,2022,english,"modern art, exhibitions, art, awards, maxwell ...",https://openlibrary.org/images/icons/avatar_bo...
4,in the shadow of trees,belgium) photobrussels festival (6th 202...,2022,french,"artistic photography, exhibitions, awards, pho...",https://openlibrary.org/images/icons/avatar_bo...


In [6]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1038 entries, 0 to 1037
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   title           1038 non-null   str  
 1   author          1038 non-null   str  
 2   published_year  1038 non-null   int64
 3   language        1038 non-null   str  
 4   subjects        1038 non-null   str  
 5   cover           1038 non-null   str  
dtypes: int64(1), str(5)
memory usage: 48.8 KB


In [7]:
df.isna().sum()

title             0
author            0
published_year    0
language          0
subjects          0
cover             0
dtype: int64

In [8]:
df.duplicated(subset=["title", "author"]).sum()

np.int64(1)

## Creating a text function

In [9]:
import re

def clean_text(s):
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)   # normalize whitespace
    s = s.replace(",", " ")      # remove comma noise
    return s

In [10]:
def safe_col(name):
    return df[name].fillna("").astype(str) if name in df.columns else pd.Series([""] * len(df))

title = safe_col("title").apply(clean_text)
author = safe_col("author").apply(clean_text)
subjects = safe_col("subjects").apply(clean_text)

df["content"] = (
    ("title: " + title + ". ") * 2 +
    ("author: " + author + ". ") * 2 +
    ("subjects: " + subjects)
)


In [11]:
#Sanity check
df[["title", "author", "subjects", "content"]].head(3)

,title,author,subjects,content
0,21st century houses: riba award-winning homes,dominic bradbury,"domestic architecture, architecture, awards, m...",title: 21st century houses: riba award-winning...
1,architecture china 2020 building with nature j...,l. xiangning,"architecture, awards, sustainable architecture...",title: architecture china 2020 building with n...
2,cyberarts 2021: international compendium prix ...,markus jandl,"computer art, awards, computer animation, prix...",title: cyberarts 2021: international compendiu...


In [12]:
df[["content"]].head(2)

,content
0,title: 21st century houses: riba award-winning...
1,title: architecture china 2020 building with n...


## Generating SBERT embeddings

#### 1. Loading the SBERT model

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [14]:
#Confirming the model works
test_vec = model.encode(["hello world"], normalize_embeddings=True)
print(test_vec.shape)

(1, 384)


#### 2. Encoding all books into embeddings

In [15]:
embeddings = model.encode(
    df["content"].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
    batch_size=32
)

print("Embeddings shape:", embeddings.shape)

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

Embeddings shape: (1038, 384)


#### 3. Sanity Check

In [16]:
embeddings[:2]

array([[ 7.52910674e-02,  2.19111815e-02,  7.01169744e-02,
         8.09179917e-02,  3.20711359e-02, -1.32779740e-02,
        -1.55584797e-01, -1.46075105e-02, -5.99961728e-02,
         3.78957503e-02, -3.55015099e-02, -2.69864891e-02,
        -2.20038742e-02, -8.79281200e-04, -8.17981176e-03,
         7.06082657e-02, -9.49885175e-02,  6.92417547e-02,
         1.16661921e-01,  1.45917132e-01, -5.99882379e-02,
         1.65331010e-02,  9.39759426e-03, -3.09497081e-02,
         6.89454153e-02,  1.61516629e-02, -3.04940734e-02,
         4.25317734e-02, -4.05225642e-02, -1.17763028e-01,
        -1.55303683e-02,  2.10854746e-02,  4.12813388e-02,
         2.98780501e-02, -5.62701561e-03,  3.51183936e-02,
         2.03228258e-02, -1.96445026e-02, -1.03739812e-03,
         2.78859027e-02, -3.51956375e-02, -3.48453149e-02,
         1.72827523e-02, -1.25780236e-03, -4.70648706e-02,
        -3.17176953e-02,  8.28487650e-02,  3.60139608e-02,
        -9.13921520e-02, -3.72657366e-02,  6.41531264e-0

## Similarity search & recommendations

#### 1. Computing cosine similarity matrix

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#### 2. Basic semantic-search recommender

In [18]:
import re

def norm(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # remove spaces + punctuation
    return s


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recommend(query, top_k=10, min_sim=0.45):
    q_raw = clean_text(query)
    q_norm = norm(q_raw)

    author_norm = df["author"].fillna("").map(norm)
    title_norm  = df["title"].fillna("").map(norm)

    author_hits = df[author_norm.str.contains(q_norm, na=False)]
    title_hits  = df[title_norm.str.contains(q_norm, na=False)]

    if len(author_hits) > 0 or len(title_hits) > 0:
        seed_idx = author_hits.index[0] if len(author_hits) > 0 else title_hits.index[0]

        seed_vec = embeddings[seed_idx].reshape(1, -1)
        sims = cosine_similarity(seed_vec, embeddings).flatten()

        top_idx = sims.argsort()[::-1]
        top_idx = [i for i in top_idx if i != seed_idx][:top_k]

        results = df.loc[top_idx, ["title","author","published_year","language","subjects","cover"]].copy()
        results["similarity"] = sims[top_idx]
        results = results.reset_index(drop=True)
        results.index += 1
        return results

    # topic-mode fallback
    q_vec = model.encode([q_raw], normalize_embeddings=True)
    sims = cosine_similarity(q_vec, embeddings).flatten()
    top_idx = sims.argsort()[::-1][:top_k]

    results = df.loc[top_idx, ["title","author","published_year","language","subjects","cover"]].copy()
    results["similarity"] = sims[top_idx]
    results = results.reset_index(drop=True)
    results.index += 1
    return results

In [20]:
# Testing with exact title name
recommend("vegetarian", top_k=10)

,title,author,published_year,language,subjects,cover,similarity
1,chasing bocuse: america's journey to the culin...,philip tessier,2017,english,"cooks, cooking, bocuse d'or (competition) (201...",https://covers.openlibrary.org/b/id/14450218-M...,0.276103
2,shokubunka no shōjiten,nichigai asoshiētsu,2021,japanese,"food habits, handbooks, manuals, awards, food ...",https://openlibrary.org/images/icons/avatar_bo...,0.274640
3,"pratos da boa lembrança: restaurantes, pratos ...","anesio fassina filho, luiz guilherme c. zancan...",2016,portuguese,"cookbooks, brazilian cooking, commemorative pl...",https://openlibrary.org/images/icons/avatar_bo...,0.189588
4,nuobei'er sheng yi jiang 2005-2015: nobel priz...,ke xue yue kan she,2016,chinese,"medical scientists, nobel prizes, physiologist...",https://openlibrary.org/images/icons/avatar_bo...,0.181066
5,premio nacional de ciencia y tecnología clodo...,c. picado t.,1997,spanish,premio nacional de ciencia y tecnología clodo...,https://covers.openlibrary.org/b/id/4934058-L.jpg,0.178125
6,bonei zion prize: the nefesh b'nefesh bonei zi...,irgun nefesh be-nefesh,2015,english,"awards, american jews, immigrants, emigration ...",https://openlibrary.org/images/icons/avatar_bo...,0.167308
7,prix offerts par la société d'agriculture de...,société d'agriculture de québec.,1986,french,"société d'agriculture de québec., agricultu...",https://covers.openlibrary.org/b/id/7095641-L.jpg,0.166688
8,prix offerts par la société d'agriculture de q...,société d'agriculture de québec.,1985,french,"quebec agricultural society., agricultural exh...",https://covers.openlibrary.org/b/id/7093636-L.jpg,0.163390
9,paul flora preis 2002-2015: peter niedertschei...,"galerie im taxispalais (innsbruck, austr...",2015,"german, italian","art, catalogs, awards, modern art, artists, au...",https://openlibrary.org/images/icons/avatar_bo...,0.161589
10,por los caminos de leloir,césar lorenzano,1994,spanish,"leloir, luis f. -- awards., leloir, luis f., b...",https://covers.openlibrary.org/b/id/2291791-L.jpg,0.156222


In [21]:
# Testing with exact title name
recommend("tonychapman", top_k=10)

,title,author,published_year,language,subjects,cover,similarity
1,architecture 05: the guide to the riba awards ...,tony chapman,2006,english,"architecture, architecture (general), awards, ...",https://covers.openlibrary.org/b/id/2059172-L.jpg,0.958589
2,architecture 07: the guide to the riba awards ...,tony chapman,2007,english,"general, reference, architecture",https://covers.openlibrary.org/b/id/3011132-L.jpg,0.924086
3,architecture 04: the guide to the riba awards ...,tony chapman,2005,english,"architecture, yearbooks, annuals, almanacs, re...",https://covers.openlibrary.org/b/id/912503-L.jpg,0.920226
4,architecture 03: the riba awards (architecture...,tony chapman,2003,english,"architecture, british isles, history - general...",https://covers.openlibrary.org/b/id/912498-L.jpg,0.905063
5,architecture 02,tony chapman,2003,english,"architecture, british isles, europe, united ki...",https://covers.openlibrary.org/b/id/2552815-L.jpg,0.799773
6,architecture 00,tony chapman,2001,english,"architecture, general, europe, united kingdom,...",https://covers.openlibrary.org/b/id/2018258-L.jpg,0.798498
7,architecture canada 2010,royal architectural institute of canada,2010,english,"modern architecture, awards, architecture",https://covers.openlibrary.org/b/id/10992635-L...,0.787645
8,architecture awards selection: selección premi...,manuel rúa garcía,2015,english,"architecture, history, awards, spain, 21st cen...",https://openlibrary.org/images/icons/avatar_bo...,0.755309
9,the thomas jefferson foundation medal in archi...,karen van lengen,2007,english,"architectural structure & design, honours & de...",https://covers.openlibrary.org/b/id/5161588-L.jpg,0.750656
10,architecture for a changing world,james steele,1992,english,aga khan award for architecture (organization)...,https://covers.openlibrary.org/b/id/4872080-L.jpg,0.740001


In [22]:
# Testing with free text search
recommend("award-winning sustainable architecture", top_k=10)

,title,author,published_year,language,subjects,cover,similarity
1,sustainable design 5: vers une nouvelle éthiqu...,marie-hélène contal,2017,"french, english","sustainable design, sustainable architecture, ...",https://openlibrary.org/images/icons/avatar_bo...,0.694919
2,sustainable design,marie-hélène contal,2016,"french, english","sustainable architecture, awards, architecture...",https://covers.openlibrary.org/b/id/12834649-L...,0.680977
3,sustainable design 8: vers une nouvelle éthiqu...,marie-hélène contal,2020,french,"sustainable architecture, awards, history, arc...",https://openlibrary.org/images/icons/avatar_bo...,0.680556
4,sustainable design 9: vers une nouvelle éthiqu...,marie-hélène contal,2022,french,"sustainable architecture, awards, history, glo...",https://openlibrary.org/images/icons/avatar_bo...,0.679100
5,sustainable design 6: vers une nouvelle éthiqu...,marie-hélène contal,2018,"french, english","sustainable design, sustainable architecture, ...",https://openlibrary.org/images/icons/avatar_bo...,0.663555
6,sustainable design: vers une nouvelle éthique ...,marie-hélène contal,2016,"french, english","sustainable architecture, awards, architecture...",https://covers.openlibrary.org/b/id/12834649-M...,0.661625
7,terrafibra architectures,"dominique gauzin-müller, aurélie vissac",2021,french,"adobe houses, exhibitions, awards, earth const...",https://openlibrary.org/images/icons/avatar_bo...,0.596240
8,architizer: the world's best architecture prac...,architizer,2021,english,"architecture, modern, 21st century, architectu...",https://covers.openlibrary.org/b/id/12419105-M...,0.593916
9,21st century houses: riba award-winning homes,dominic bradbury,2022,english,"domestic architecture, architecture, awards, m...",https://openlibrary.org/images/icons/avatar_bo...,0.593040
10,best highrises 2020/21: the international high...,"peter cachola schmal, peter körner, stefanie l...",2021,"english, german","high-rise apartment buildings, design and cons...",https://openlibrary.org/images/icons/avatar_bo...,0.586877


#### 3. Adding filters (language + year)

In [23]:
def recommend_filtered(query, top_k=10, language=None, min_year=None, max_year=None):
    query = clean_text(query)
    q_vec = model.encode([query], normalize_embeddings=True)

    sims = cosine_similarity(q_vec, embeddings).flatten()

    df_sim = df.copy()
    df_sim["similarity"] = sims

    # filters
    if language:
        df_sim = df_sim[df_sim["language"].str.lower() == language.lower()]

    if min_year is not None:
        df_sim = df_sim[df_sim["published_year"] >= min_year]

    if max_year is not None:
        df_sim = df_sim[df_sim["published_year"] <= max_year]

    results = (
        df_sim
        .sort_values("similarity", ascending=False)
        .head(top_k)
        [["title", "author", "published_year", "language", "subjects", "cover", "similarity"]]
    )
    
    results = results.reset_index(drop=True)
    results.index = results.index + 1

    return results

In [24]:
#Testing with filters
recommend_filtered(
    "cooking",
    language="english",
    min_year=2017,
    top_k=10
)


,title,author,published_year,language,subjects,cover,similarity
1,chasing bocuse: america's journey to the culin...,philip tessier,2017,english,"cooks, cooking, bocuse d'or (competition) (201...",https://covers.openlibrary.org/b/id/14450218-M...,0.427489
2,complex management systems and the shingo mode...,gerhard j. plenert,2019,english,"production management, lean manufacturing, man...",https://openlibrary.org/images/icons/avatar_bo...,0.235670
3,how to manage a writing competition: a handboo...,mahmoud mansi,2019,english,"awards, performance standards, competitions, c...",https://covers.openlibrary.org/b/id/15094610-M...,0.141636
4,best american magazine writing 2017,"sid holt, the american so editors",2018,english,"american prose literature, journalism, awards",https://openlibrary.org/images/icons/avatar_bo...,0.127133
5,not just your face honey,"stefanie moshammer, andreas prinzing",2019,english,"exhibitions, artistic photography, pictorial w...",https://covers.openlibrary.org/b/id/13909182-M...,0.124255
6,peace and me,"ali winter, mickael el fathi",2018,english,"pacifists, biography, juvenile literature, nob...",https://covers.openlibrary.org/b/id/12477144-M...,0.116687
7,beazley designs of the year 2019,"beatrice galilee, maria mclintock",2019,english,"design, awards, industrial design",https://openlibrary.org/images/icons/avatar_bo...,0.112414
8,baloise art,"david campany, martin schwander, manuela ammer...",2020,english,"art, awards, modern art, art collections, art ...",https://openlibrary.org/images/icons/avatar_bo...,0.111765
9,trophy night,ryan howard,2017,english,"juvenile fiction, awards, baseball stories, ch...",https://openlibrary.org/images/icons/avatar_bo...,0.109343
10,best american magazine writing 2019,sid holt,2019,english,"american prose literature, journalism, awards",https://openlibrary.org/images/icons/avatar_bo...,0.108712


In [25]:
#Testing with filters
recommend_filtered(
    "modern architecture houses",
    language="english",
    min_year=2015,
    top_k=10
)


,title,author,published_year,language,subjects,cover,similarity
1,21st century houses: riba award-winning homes,dominic bradbury,2022,english,"domestic architecture, architecture, awards, m...",https://openlibrary.org/images/icons/avatar_bo...,0.663598
2,out of scale,"marc manack, linda reeder",2015,english,"architecture, united states, awards, architect...",https://openlibrary.org/images/icons/avatar_bo...,0.524697
3,architectures on the waterfront,"ivan blasi, anna sala giralt",2019,english,"waterfronts, exhibitions, city planning, mies ...",https://openlibrary.org/images/icons/avatar_bo...,0.524348
4,atlas: european union prize for contemporary a...,"celia marín vega, marina romero",2016,english,"architecture, awards, history, mies van der ro...",https://openlibrary.org/images/icons/avatar_bo...,0.505293
5,young talent architectture award 2018,"ivan blasi, anna sala giralt",2018,english,"architecture, history, designs and plans, dome...",https://openlibrary.org/images/icons/avatar_bo...,0.493510
6,euregional prize for architecture 2006-2015,"andrea croé, jeanine ruijters, minnou veders-m...",2015,english,"euregional prize for architecture, architectur...",https://openlibrary.org/images/icons/avatar_bo...,0.492002
7,eu mies award 19: european union prize for con...,"ivan blasi, anna sala giralt",2019,english,"architecture, awards, history, mies van der ro...",https://openlibrary.org/images/icons/avatar_bo...,0.489714
8,young talent architecture award 2016,"ivan blasi, anna sala giralt",2018,english,"architecture, history, designs and plans, dome...",https://openlibrary.org/images/icons/avatar_bo...,0.489287
9,eu mies award 17: european union prize for con...,"ivan blasi, anna sala giralt",2017,english,"architecture, awards, history, mies van der ro...",https://openlibrary.org/images/icons/avatar_bo...,0.488783
10,architizer: the world's best architecture prac...,architizer,2021,english,"architecture, modern, 21st century, architectu...",https://covers.openlibrary.org/b/id/12419105-M...,0.486567


#### Saving embeddings & data

In [26]:
import numpy as np

np.save("book_embeddings.npy", embeddings)
df.to_csv("books_with_content.csv", index=False)


In [27]:
embeddings = np.load("book_embeddings.npy")
df = pd.read_csv("books_with_content.csv")